In [1]:
from fastai.vision.all import *

In [2]:
path = untar_data(URLs.PETS)
Path.BASE_PATH = path

In [13]:
(path/'images').ls()[:4]

(#4) [Path('images/miniature_pinscher_199.jpg'),Path('images/newfoundland_183.jpg'),Path('images/pomeranian_90.jpg'),Path('images/pomeranian_102.jpg')]

In [11]:
with open(path/'annotations'/'list.txt') as f:
    print(f.read())

#Image CLASS-ID SPECIES BREED ID
#ID: 1:37 Class ids
#SPECIES: 1:Cat 2:Dog
#BREED ID: 1-25:Cat 1:12:Dog
#All images with 1st letter as captial are cat images
#images with small first letter are dog images
Abyssinian_100 1 1 1
Abyssinian_101 1 1 1
Abyssinian_102 1 1 1
Abyssinian_103 1 1 1
Abyssinian_104 1 1 1
Abyssinian_105 1 1 1
Abyssinian_106 1 1 1
Abyssinian_107 1 1 1
Abyssinian_108 1 1 1
Abyssinian_109 1 1 1
Abyssinian_10 1 1 1
Abyssinian_110 1 1 1
Abyssinian_111 1 1 1
Abyssinian_112 1 1 1
Abyssinian_113 1 1 1
Abyssinian_114 1 1 1
Abyssinian_115 1 1 1
Abyssinian_116 1 1 1
Abyssinian_117 1 1 1
Abyssinian_118 1 1 1
Abyssinian_119 1 1 1
Abyssinian_11 1 1 1
Abyssinian_120 1 1 1
Abyssinian_121 1 1 1
Abyssinian_122 1 1 1
Abyssinian_123 1 1 1
Abyssinian_124 1 1 1
Abyssinian_125 1 1 1
Abyssinian_126 1 1 1
Abyssinian_127 1 1 1
Abyssinian_128 1 1 1
Abyssinian_129 1 1 1
Abyssinian_12 1 1 1
Abyssinian_130 1 1 1
Abyssinian_131 1 1 1
Abyssinian_132 1 1 1
Abyssinian_133 1 1 1
Abyssinian_134 1 1 1


In [91]:
three = torch.stack([tensor(Image.open(im)) for im in (path/'valid'/'3').ls()])/255.
seven = torch.stack([tensor(Image.open(im)) for im in (path/'valid'/'7').ls()])/255.
X_valid = torch.cat([three, seven]).reshape(-1, 28*28)
y_valid = torch.tensor([1]*len(three) + [0]*len(seven)).reshape(-1, 1)
dset_valid = list(zip(X_valid, y_valid))

In [77]:
def create_dataloader(data, shuffle=False):
    three = torch.stack([tensor(Image.open(im)) for im in (path/data/'3').ls()])/255.
    seven = torch.stack([tensor(Image.open(im)) for im in (path/data/'7').ls()])/255.
    X_train = torch.cat([three, seven]).reshape(-1, 28*28)
    y_train = torch.tensor([1]*len(three) + [0]*len(seven)).reshape(-1, 1)
    dset = list(zip(X_train, y_train))
    return DataLoader(dset, batch_size=256, shuffle=shuffle)

In [104]:
def init_params():
    std = 1.0
    weight = (torch.randn(28*28, 1)*std).requires_grad_()
    bias = (torch.randn(1)*std).requires_grad_()
    return [weight, bias]

In [105]:
def linear1(x):
    return x@weight + bias

In [106]:
def calc_loss(pred, y):
    pred = (lambda x: 1/(1 + torch.exp(-x)))(pred)
    return torch.where(y==1, 1 - pred, pred).mean()

In [107]:
def calc_accuracy(pred, y):
    pred = (lambda x: 1/(1 + torch.exp(-x)))(pred)
    return ((pred>0.5)==y).float().mean()

In [108]:
def train_epoch(dl, lr):
    acc = calc_accuracy(linear1(X_valid), y_valid)
    print(acc)
    for X, y in dl:
        pred = linear1(X)
        loss = calc_loss(pred, y)
        loss.backward()
        for p in weight, bias:
            p.data -= p.grad*lr
            p.grad = None

In [109]:
dl = create_dataloader('train', shuffle=True)
dl_valid = create_dataloader('valid')
lr = 1.

In [124]:
weight, bias = init_params()

for _ in range(30):
    train_epoch(dl, lr)

tensor(0.5015)
tensor(0.9151)
tensor(0.9362)
tensor(0.9436)
tensor(0.9504)
tensor(0.9519)
tensor(0.9539)
tensor(0.9539)
tensor(0.9544)
tensor(0.9563)
tensor(0.9578)
tensor(0.9573)
tensor(0.9588)
tensor(0.9612)
tensor(0.9627)
tensor(0.9632)
tensor(0.9647)
tensor(0.9642)
tensor(0.9652)
tensor(0.9652)
tensor(0.9657)
tensor(0.9657)
tensor(0.9666)
tensor(0.9671)
tensor(0.9676)
tensor(0.9681)
tensor(0.9691)
tensor(0.9701)
tensor(0.9701)
tensor(0.9706)


In [126]:
dls = DataLoaders(dl, dl_valid)
model = nn.Sequential(
    nn.Linear(28*28, 30),
    nn.ReLU(),
    nn.Linear(30, 1))
params = list(model.parameters())

learn = Learner(dls, model, loss_func=calc_loss, opt_func=SGD, metrics=calc_accuracy)
learn.fit(10, 1)

epoch,train_loss,valid_loss,calc_accuracy,time
0,0.054885,0.031478,0.973013,00:00
1,0.030741,0.024967,0.977920,00:00
2,0.022546,0.022389,0.979392,00:00
3,0.018850,0.020173,0.984298,00:00
4,0.016454,0.018782,0.983808,00:00
5,0.015628,0.018900,0.983808,00:00
6,0.014786,0.018660,0.982336,00:00
7,0.013849,0.017598,0.984789,00:00
8,0.013092,0.017370,0.984789,00:00
9,0.013077,0.017571,0.982336,00:00


In [128]:
dls = ImageDataLoaders.from_folder(path)
learn = vision_learner(dls, resnet18, pretrained=False,
                    loss_func=F.cross_entropy, metrics=accuracy)
learn.fit_one_cycle(1, 0.1)

epoch,train_loss,valid_loss,accuracy,time
0,0.064423,0.016916,0.997056,00:17
